In [1]:
from src.data._dataobject import DataObject
from src.data.types._date import NSDate
from src.data.types._datetime import DateTime
from src.data.types._text import String, NSString
from src.data.types._int import Int
from src.data.types._phone_number import PhoneNumber
from src.data.types._id import MongoId, Id, StrId
from src.data.types._time import Time
from src.connections._elasticsearch import Elasticsearch
from src.api._api import Api

In [2]:

class User(DataObject):
    __conn__ = Elasticsearch
    first_name = NSString(min_length=3, json_field='firstName')
    last_name = NSString(json_field='lastName')
    mobile = PhoneNumber()
    id = StrId()
    created = DateTime(default='now')
    birthday=NSDate()
    favourite_time=Time()

In [3]:
User.drop_table()

True

In [4]:
User.create_table()

True

In [5]:
u = User(
    first_name='Frodo',
    last_name='Baggins',
    mobile='+39 375 57 44 512',
    birthday='1988-07-25',
    favourite_time='18:00',
)

In [6]:
u.add_to_db()

True

In [7]:
u.to_dict()

{'birthday': '1988-07-25',
 'created': '2022-02-15T18:26:40.722955',
 'favourite_time': '18:00:00.000000',
 'first_name': 'Frodo',
 '_id': '7ARs_n4BxgfpqKEoE0cT',
 'last_name': 'Baggins',
 'mobile': '+39 375 57 44 512'}

In [8]:
u.first_name = 'Frodo Jr.'

In [9]:
u.add_to_db()

True

In [10]:
u.to_dict()

{'birthday': '1988-07-25',
 'created': '2022-02-15T18:26:40.722955',
 'favourite_time': '18:00:00.000000',
 'first_name': 'Frodo Jr.',
 '_id': '7ARs_n4BxgfpqKEoE0cT',
 'last_name': 'Baggins',
 'mobile': '+39 375 57 44 512'}

In [11]:
users = User.get_from_db()

In [12]:
for user in users:
    print(user.to_json())

In [13]:
api = Api()
api.add_entity(User)

In [14]:
api.start()

RuntimeError: This event loop is already running

In [ ]:
# api.stop()

NameError: name 'api' is not defined